# 8 Queens

### Introduction

<figure>
<img src="resources/eight_queens_moves.png", width=300 align="right">
    <figcaption></figcaption>
</figure>

The [Eight Queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a famous puzzle that has been studied extensively in- and outside of computer science. It was first published in the chess magazine _Schach_ in 1848. 

The problem can be formulated as follows: 

_"Place 8 queens on a regular (8x8) chess board such that no queen attacks any other queen."_

A queen in the game of chess can move horizontally, vertically, and diagonally. The puzzle can be solved by hand (and even [Carl Friedrich Gauss](https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss) studied it back in 1850).

The EightQueensState class below, as well as the methods defined, should prove a helpful start for a Genetic Algorithms approach. However, you are welcome to change as little or as much of the code as is useful.

In [14]:
import numpy as np

class EightQueensState:
    """This class represents a board in the eight queens puzzle"""
    def __init__(self, n, state=None):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        if state is None:
            self.n = n
            state = np.random.randint(0, n, n)
        else:
            self.n = len(state)
        
        self.state = state        
        self._fitness = ((self.n*(self.n-1))-self.cost())/2

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))

    def cost(self):
        """Calculates the number of pairs attacking"""
        count = 0
        for i in range(len(self.state) - 1):
            # for each queen, look in columns to the right
            # add one to the count if there is another queen in the same row
            count += (self.state[i] == np.array(self.state[i + 1:])).sum()

            # add one to the count for each queen on the upper or lower diagonal
            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += (np.array(self.state[i + 1:]) == upper_diagonal).sum()
            count += (np.array(self.state[i + 1:]) == lower_diagonal).sum()
        return count
    
    def fitness(self):
        """Return the fitness valuation (n*(n-1)-cost)/2"""
        return self._fitness
    
    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood
    
    def mutate(self):
        """Mutate a letter of the baby DNA and hope for the best"""
        new_state = self.state.copy()
        v = randint(0, self.n-1)
        new_state[v] = randint(0, self.n-1)
        return EightQueensState(self.n, state=new_state), v

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)

    def is_goal(self):
        return self.cost() == 0

    def __str__(self):
        if self.is_goal():
            return f"Goal state! {self.state}"
        else:
            return f"{self.state} cost {self.cost()}"
        


In [15]:
from random import random, randint, shuffle, choices

class EightQueensPopulation:
    """This class represents a generation of puzzle game states"""
    
    def __init__(self, queen_count, population=None, size=None, generation=0):
        """
        If no population is passed at init the population will be created based
        on the size parameter.
        
        :param state: pass in a list of EightQueensState objects
        """        
        if not population:
            self._pop=[]
            if size:
                for i in range(size):
                    self._pop.append(EightQueensState(queen_count))

        else:
            self._pop = population
        self._gen = generation
        self.n = queen_count
        self._log = []
    
    def print_state(self, dump_pop=False, dump_log=False):
        """
        Prints debug output
        
        :param dump_pop: prints the whole population
        :param dump_log: prints the step log including parents, mutations
        """
        avg_fit = sum([i.fitness() for i in self._pop])/len(self._pop)
        probs = self.probabilities()
        print("Generation {}, avg fit {}".format(self._gen, avg_fit))       
        if dump_pop:
            for i in range(len(self._pop)):
                print("{}, {}".format(self._pop[i], probs[i]*100))
        if dump_log:
            for t in self._log:
                if len(t) == 4:
                    print("{} + {} at {} = {}".format(t[0],t[1],t[2],t[3]))
                elif len(t)==1:
                    print("Mutated to {}".format(t[0]))
        
    def grow(self, i):
        """Adds DNA to the population"""
        self._pop.append(i)
    
    def size(self):
        """Returns the size of population"""
        return len(self._pop)
    
    @staticmethod
    def norm(data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    
    def max_fitness(self):
        """Returns the maximum fitness score"""
        return (self.n*(self.n-1))/2
    
    def probabilities(self):    
        # fit_sum = sum([i.fitness() for i in self._pop])
        # normalized = EightQueensPopulation.norm(np.array([i.fitness()/fit_sum for i in self._pop])).tolist()        
        # return [(p, i) for p, i in zip(normalized, self._pop)]
        return [i.fitness()/self.max_fitness() for i in self._pop]
    
    def cull(self, candidates, cutoff=0.2):
        """Cull worst solutions out of population"""
        return [(p, i) for p,i in candidates if p >= cutoff]        
    
    def pick_n(self, culling = True, n=2):
        """Random weighted pick from the population"""
        
        return choices(self._pop, weights=[i.fitness() for i in self._pop], k=n)
    
    def log(self, t):
        self._log.append(t)

    def offspring(self, parents):
        """Create a new DNA by merging two parents' DNA codes at random point"""
        n = len(parents[0].state)
        inx = randint(0, n)
        left = parents[0].state.copy()[0:inx]
        right = parents[1].state.copy()[inx:n]
        return EightQueensState(self.n, state=np.concatenate([left,right])), inx
        

In [16]:
from random import random, randint, shuffle

N=8                  # number of queens
N_POPULATION=100     # size of population
P_MUTATION=0.03      # probability of mutation

def genetic_algo():

    gen_i = 0        # Generation count
    old_gen = EightQueensPopulation(N, size = N_POPULATION, generation=gen_i)

    not_found = True
    while not_found:        
        next_gen=EightQueensPopulation(N, generation=gen_i+1)

        for i in range(N_POPULATION):
            
            parents = old_gen.pick_n(culling=False)
            baby, inx = next_gen.offspring(parents)

            next_gen.log((parents[0], parents[1], inx, baby))
            
            if random() < P_MUTATION:
                baby, inx = baby.mutate()
                next_gen.log((baby, inx))                         

            if(next_gen.size() < old_gen.size()):
                next_gen.grow(baby)
            
            not_found = not baby.is_goal()
            
            if not not_found:
                # Found solution:
                print(baby)
                break
        old_gen=next_gen
        gen_i += 1  
        old_gen.print_state(dump_pop=False)        

genetic_algo()





Generation 1, avg fit 24.03
Generation 2, avg fit 24.12
Generation 3, avg fit 24.415
Generation 4, avg fit 24.16
Generation 5, avg fit 24.26
Generation 6, avg fit 24.385
Generation 7, avg fit 24.47
Generation 8, avg fit 24.325
Generation 9, avg fit 24.385
Generation 10, avg fit 24.295
Generation 11, avg fit 24.16
Generation 12, avg fit 24.205
Generation 13, avg fit 24.115
Generation 14, avg fit 24.42
Generation 15, avg fit 24.355
Generation 16, avg fit 24.305
Generation 17, avg fit 24.22
Generation 18, avg fit 24.35
Generation 19, avg fit 24.27
Generation 20, avg fit 24.135
Generation 21, avg fit 24.18
Generation 22, avg fit 24.165
Generation 23, avg fit 24.17
Generation 24, avg fit 24.105
Generation 25, avg fit 24.05
Generation 26, avg fit 24.23
Generation 27, avg fit 24.39
Generation 28, avg fit 24.485
Generation 29, avg fit 24.41
Generation 30, avg fit 24.365
Generation 31, avg fit 24.39
Generation 32, avg fit 24.37
Generation 33, avg fit 24.395
Generation 34, avg fit 24.565
Generat

Generation 275, avg fit 25.55
Generation 276, avg fit 25.575
Generation 277, avg fit 25.525
Generation 278, avg fit 25.58
Generation 279, avg fit 25.555
Generation 280, avg fit 25.545
Generation 281, avg fit 25.615
Generation 282, avg fit 25.73
Generation 283, avg fit 25.695
Generation 284, avg fit 25.66
Generation 285, avg fit 25.635
Generation 286, avg fit 25.71
Generation 287, avg fit 25.665
Generation 288, avg fit 25.55
Generation 289, avg fit 25.59
Generation 290, avg fit 25.58
Generation 291, avg fit 25.585
Generation 292, avg fit 25.65
Generation 293, avg fit 25.64
Generation 294, avg fit 25.58
Generation 295, avg fit 25.63
Generation 296, avg fit 25.65
Generation 297, avg fit 25.68
Generation 298, avg fit 25.615
Generation 299, avg fit 25.49
Generation 300, avg fit 25.565
Generation 301, avg fit 25.52
Generation 302, avg fit 25.45
Generation 303, avg fit 25.43
Generation 304, avg fit 25.31
Generation 305, avg fit 25.355
Generation 306, avg fit 25.43
Generation 307, avg fit 25.5

Generation 552, avg fit 25.92
Generation 553, avg fit 25.98
Generation 554, avg fit 25.98
Generation 555, avg fit 26.015
Generation 556, avg fit 25.99
Generation 557, avg fit 26.015
Generation 558, avg fit 26.055
Generation 559, avg fit 25.96
Generation 560, avg fit 25.975
Generation 561, avg fit 25.995
Generation 562, avg fit 25.785
Generation 563, avg fit 25.835
Generation 564, avg fit 25.875
Generation 565, avg fit 25.76
Generation 566, avg fit 25.7
Generation 567, avg fit 25.795
Generation 568, avg fit 25.79
Generation 569, avg fit 25.72
Generation 570, avg fit 25.76
Generation 571, avg fit 25.705
Generation 572, avg fit 25.74
Generation 573, avg fit 25.785
Generation 574, avg fit 25.785
Generation 575, avg fit 25.725
Generation 576, avg fit 25.795
Generation 577, avg fit 25.81
Generation 578, avg fit 25.92
Generation 579, avg fit 26.015
Generation 580, avg fit 26.055
Generation 581, avg fit 26.0
Generation 582, avg fit 26.08
Generation 583, avg fit 26.125
Generation 584, avg fit 2

Generation 823, avg fit 26.305
Generation 824, avg fit 26.345
Generation 825, avg fit 26.49
Generation 826, avg fit 26.525
Generation 827, avg fit 26.44
Generation 828, avg fit 26.385
Generation 829, avg fit 26.345
Generation 830, avg fit 26.385
Generation 831, avg fit 26.285
Generation 832, avg fit 26.21
Generation 833, avg fit 26.225
Generation 834, avg fit 26.24
Generation 835, avg fit 26.21
Generation 836, avg fit 26.165
Generation 837, avg fit 26.16
Generation 838, avg fit 26.12
Generation 839, avg fit 26.195
Generation 840, avg fit 26.255
Generation 841, avg fit 26.24
Generation 842, avg fit 26.145
Generation 843, avg fit 26.23
Generation 844, avg fit 26.32
Generation 845, avg fit 26.325
Generation 846, avg fit 26.25
Generation 847, avg fit 26.22
Generation 848, avg fit 26.195
Generation 849, avg fit 26.08
Generation 850, avg fit 26.14
Generation 851, avg fit 26.145
Generation 852, avg fit 25.94
Generation 853, avg fit 25.99
Generation 854, avg fit 26.075
Generation 855, avg fit 